In [1]:
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
import math
x=80
sigmoid = 1 / (1 + math.exp(-x))
sigmoid

1.0

In [10]:
import math
var=[80,8,1,.1,-.1,-1,-8,-80]
for x in var:
    sigmoid = 1 / (1 + math.exp(-x))
    print(sigmoid)

1.0
0.9996646498695336
0.7310585786300049
0.52497918747894
0.47502081252106
0.2689414213699951
0.0003353501304664781
1.8048513878454153e-35


In [2]:
20%10

0

In [1]:
import pygame
import time
import random
import math
import numpy
import pandas as pd
import colorsys
import copy

from pygame import gfxdraw

# Create the pandas DataFrame with column name is provided explicitly
def main():
	pygame.init()
	game_width = 800 #1600
	game_height = 600 #900
	white = (255,255,255)
	black = (0,0,0)
	red = (255,0,0)
	green = (0,255,0)
	blue = (0,0,255)
	fps = 60
	size = 5
	mutation_rate = 0.2
	steering_weights = 0.05
	perception_radius_mutation_range = 30
	reproduction_rate = 0.000005 #0.0005
	initial_perception_radius = 100
	boundary_size = 50
	max_vel = 10
	max_accel = 2
	attraction_rate = 0.2 #0.02
	#global_enviromental_coeff_friction = .25
	health = 100
	max_poison = 50
	nutrition = [20, -80]
	all_bots = []
	bots = []
	herbivore={}
	food = []
	poison = []
	oldest_ever = 0
	oldest_ever_dna = []
	gameDisplay = pygame.display.set_mode((game_width, game_height))
	clock = pygame.time.Clock()
	unique_names=[1]
	things_that_live = pygame.sprite.Group()


	##generate sprite
	##arrays for shapes
	robot = [[0, 0, 0, 0],
			[0, 1, 1, 1],
			[0, 1, 2, 2],
			[0, 0, 1, 2],
			[0, 0, 0, 2],
			[1, 1, 1, 2],
			[0, 1, 1, 2],
			[0, 0, 0, 2],
			[0, 0, 0, 2],
			[0, 1, 2, 2],
			[1, 1, 0, 0]]

	space = [[0, 0, 0, 0, 0, 0],
			[0, 0, 0, 0, 1, 1],
			[0, 0, 0, 0, 1,-1],
			[0, 0, 0, 1, 1,-1],
			[0, 0, 0, 1, 1,-1],
			[0, 0, 1, 1, 1,-1],
			[0, 1, 1, 1, 2, 2],
			[0, 1, 1, 1, 2, 2],
			[0, 1, 1, 1, 2, 2],
			[0, 1, 1, 1, 1,-1],
			[0, 0, 0, 1, 1, 1],
			[0, 0, 0, 0, 0, 0]]

	dragon_top = [[0, 0, 0, 0, 0, 0, 0, 0],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 1, 1, 1, 1],
				[0, 0, 1, 1, 1, 1, 1, 1],
				[0, 0, 1, 1, 1, 1, 1, 1],
				[0, 1, 1, 0, 0, 0, 1, 1],
				[0, 1, 0, 0, 0, 0, 1, 1],
				[0, 1, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 1, 1],
				[0, 0, 0, 0, 0, 0, 0, 0]]
				
	animal_list = [robot,space,dragon_top]

	def generate(mask, color_variations=0.2, colored=True, brightness_noise=0.3, edge_brightness=0.3,
				saturation=0.2, mirror=False):

		new_mask = copy.deepcopy(mask)
		height = len(new_mask)
		width = len(new_mask[0])

		is_vertical_gradient = random.random() > 0.5
		saturation = max(min(random.random() * saturation, 1), 0)
		hue = random.random()

	#### generate body
		for x in range(width):
			for y in range(height):
				if new_mask[y][x] == 1:
					new_mask[y][x] = round(random.random() > 0.5)
				elif new_mask[y][x] == 2:
					if random.random() > 0.5:
						new_mask[y][x] = 1
					else:
						new_mask[y][x] = -1
	#### if mirroring
		if mirror:
			for i in range(len(new_mask)):
				new_mask[i].extend(new_mask[i][::-1])
			height = len(new_mask)
			width = len(new_mask[0])

	#### generate edges
		for x in range(width):
			for y in range(height):
				if new_mask[y][x] == 1:
					if x - 1 >= 0 and new_mask[y][x - 1] == 0:
						new_mask[y][x - 1] = -1
					if x + 1 < width and new_mask[y][x + 1] == 0:
						new_mask[y][x + 1] = -1
					if y - 1 >= 0 and new_mask[y - 1][x] == 0:
						new_mask[y - 1][x] = -1
					if y + 1 < height and new_mask[y + 1][x] == 0:
						new_mask[y + 1][x] = -1


		new_mask = _apply_colors(new_mask, is_vertical_gradient, color_variations, colored, brightness_noise, saturation,
								edge_brightness, hue, height, width)

		return new_mask

	def _apply_colors(mask, is_vertical_gradient, color_variations, colored, brightness_noise, saturation,
					edge_brightness, hue, height, width):
		""" Apply colors to the mask """
		if is_vertical_gradient:
			ulen = height
			vlen = width
		else:
			ulen = height
			vlen = width

		# Fill colors randomly
		for u in range(ulen):
			# Create a non-uniform random number between 0 and 1 (lower numbers more likely)
			is_new_color = abs((random.random() * 2 - 1)
							+ (random.random() * 2 - 1)
							+ (random.random() * 2 - 1)) / 3

			if is_new_color > (1 - color_variations):
				hue = random.random()

			for v in range(vlen):
				if is_vertical_gradient:
					val = mask[u][v]
				else:
					val = mask[u][v]

				rgb = {"r": 1, "g": 1, "b": 1}

				if val != 0:
					if colored:
						brightness = (math.sin((u / ulen) * math.pi) *
									(1 - brightness_noise) + random.random() * brightness_noise)
						rgb_vals = colorsys.hls_to_rgb(hue, brightness, saturation)
						rgb['r'] = rgb_vals[0]
						rgb['g'] = rgb_vals[1]
						rgb['b'] = rgb_vals[2]

						if val == -1:
							rgb['r'] *= edge_brightness
							rgb['g'] *= edge_brightness
							rgb['b'] *= edge_brightness

					else:
						if val == -1:
							rgb = {"r": 0, "g": 0, "b": 0}

				mask[u][v] = (round(rgb['r'] * 255),
								round(rgb['g'] * 255),
								round(rgb['b'] * 255))

		return mask


	def update_damage_color():
		percent_health = bot.health/health
		lerped_colour = (max(min((1-percent_health)*255,255),0), max(min(percent_health*255,255),0), 0)
		return(lerped_colour)

	def magnitude_calc(vector):
		x = 0
		for i in vector:
			x += i**2
		magnitude = x**0.5
		return(magnitude)

	def normalise(vector):
		magnitude = magnitude_calc(vector)
		if magnitude != 0:
			vector = vector/magnitude
		return(vector)

	def rot_image_list(self):
		temp_rot_list=[]
		for i in range(0,360,18):
			temp_rot = pygame.transform.rotate(self.sprite_surface_og,-(i)).copy()
			temp_rot.set_colorkey((255, 255, 255))
			temp_rot_list.append(temp_rot)
		return temp_rot_list


	def leg_colors(self):
					if (self.steer_food / attraction_rate)>0:
						food_color=(0,min(255,255*(self.steer_food/attraction_rate)),0)
					else:
						food_color=(min(255,255*abs(self.steer_food/attraction_rate)),0,0)

					if self.steer_poison / attraction_rate >0:
						poison_color=(0,0,min(255*self.steer_poison/attraction_rate,255))
					else:
						poison_color=(min(255*abs(self.steer_poison/attraction_rate),255),0,0)

					return food_color, poison_color

	class create_bot(): #How to input dna????
		def __init__(self, x, y, dna=False):

			self.name=unique_names[-1]
			self.core_position = numpy.array([x,y], dtype='float64')
			self.direction_vector = numpy.array([random.uniform(-max_vel,max_vel),random.uniform(-max_vel,max_vel)], dtype='float64')
			self.current_acceleration = numpy.array([1,1], dtype='float64')
			self.colour = green
			#print(magnitude_calc(self.direction_vector)*10)
			self.max_health = random.uniform(60,120)
			self.health = self.max_health
			self.max_vel = int(random.uniform(0,3))
			self.size = self.health/20
			self.accel_scaler = random.uniform(0,max_accel)
			self.max_force = numpy.array([self.accel_scaler, self.accel_scaler],dtype='float64')/self.size
			self.age = 1
			self.ate_counter = 0
			self.reproduction_count = 0
			self.direction = numpy.array([0, 0], dtype='float64')
			self.steer_food = random.uniform(-attraction_rate, attraction_rate)
			self.steer_poison = random.uniform(-attraction_rate, attraction_rate)
			self.percept_food = random.uniform(0, initial_perception_radius)
			self.percept_poison = random.uniform(0, initial_perception_radius)
			self.food_color, self.poison_color = leg_colors(self)
			self.image_array = generate(random.choice(animal_list), color_variations=random.random(), brightness_noise=random.random(), edge_brightness=random.random(), saturation=random.random(), colored=True, mirror=True)
			self.sprite_surface_og = pygame.transform.scale(pygame.surfarray.make_surface(numpy.array(self.image_array)),((len(self.image_array[0])*(self.health//30)),(len(self.image_array)*(self.health//30))))
			self.sprite_mask_og = pygame.mask.from_surface(self.sprite_surface_og)
			self.offset_surface_for_render_og = [(pygame.Surface.get_width(self.sprite_surface_og))/2,-(pygame.Surface.get_height(self.sprite_surface_og))/2]

			if dna != False:
				self.max_health = dna[4]
				self.health = self.max_health
				self.max_vel = dna[5]
				self.size = self.health/20
				self.accel_scaler = dna[7]
				self.max_force = dna[6]
				self.steer_food = dna[0]
				self.steer_poison = dna[1]
				self.percept_food = dna[2]
				self.percept_poison = dna[3]
				self.food_color, self.poison_color = leg_colors(self)
				self.image_array = dna[8]
				self.sprite_surface_og = dna[9]
				self.sprite_mask_og = [10]
				self.offset_surface_for_render_og = [11]


				if random.random() < mutation_rate:
					self.steer_food 	= self.steer_food + random.uniform(-steering_weights, steering_weights)
					self.steer_poison 	= self.steer_poison + random.uniform(-steering_weights, steering_weights)
					self.percept_food 	= self.percept_food + random.uniform(-perception_radius_mutation_range, perception_radius_mutation_range)
					self.percept_poison = self.percept_poison + random.uniform(-perception_radius_mutation_range, perception_radius_mutation_range)
					self.max_health 	= self.max_health + random.uniform(-20,20)
					self.max_vel 		= self.max_vel + random.uniform(-2,2)
					self.max_force 		= self.max_force + random.uniform(-self.accel_scaler,self.accel_scaler)
					self.accel_scaler 	= self.accel_scaler + random.uniform(-max_accel,max_accel)
			
			self.sprite_surface = []
			self.sprite_surface = rot_image_list(self)

			unique_names.append(int(unique_names[-1])+1)
			self.dna = [self.steer_food, #(steering weight for food -> attract/repel)
						self.steer_poison, #(steering weight for poison -> attract/repel)
						self.percept_food, #(perception_raidus for food -> range)
						self.percept_poison, #(perception_raidus for food -> range)
						self.max_health,
						self.max_vel,
						self.max_force,
						self.accel_scaler,
						self.image_array,
						self.sprite_surface_og,
						self.sprite_mask_og,
						self.offset_surface_for_render_og,
						]


###########################################
			#dna=(steering_weights(food,poison: attract/repel),perception_radius(food,poison))
			#print(self.dna)

		# def rot(self,angle):
		# 	og_surf=self.sprite_surface
		# 	surf = og_surf
		# 	rect = surf.get_rect(center=self.core_position-self.offset_surface_for_render)
		# 	angle = 0
		# 	change_angle = angle
		# 	surf = pygame.transform.rotate(og_surf,angle)
		# 	angle += 

		def update(self):
			self.direction_vector += self.current_acceleration

			self.direction_vector = normalise(self.direction_vector)*self.max_vel

			self.core_position += self.direction_vector
			#exp_friction = self.size * (-self.direction_vector) * global_enviromental_coeff_friction
			#self.current_acceleration = exp_friction
			self.current_acceleration *= 0
			#damage every update to force eating
			if self.max_vel<1:
				self.health -= .5
			self.health -= (0.15 * magnitude_calc(self.direction_vector))
			self.colour = update_damage_color()
			self.health = min(self.max_health, self.health)
			# if self.age % 1000 == 0:
			# 	print(self.age, self.dna)
			self.age += 1

		def reproduce(self):
			for bot in bots:
				if bot.ate_counter//10>0:
					bot.ate_counter = 0
					bots.append(create_bot(self.core_position[0], self.core_position[1], self.dna))
					#herbivore.append()
					bot.reproduction_count += 1
			# if random.random() < reproduction_rate:
			# 	bots.append(create_bot(self.core_position[0], self.core_position[1], self.dna))


		def return_true_if_dead(self):
			if self.health > 0:
				return(False)
			else:
				#when dies turns to food?
				if self.core_position[0] < game_width - boundary_size and self.core_position[0] > boundary_size and self.core_position[1] < game_height - boundary_size and self.core_position[1] > boundary_size:
					food.append(self.core_position)
				return(True)

		def apply_force(self, force):
			self.current_acceleration += force
			
		def seek(self, target):
			desired_vel = numpy.add(target, -self.core_position)
			desired_vel = normalise(desired_vel)*self.max_vel
			steering_force = numpy.add(desired_vel, -self.direction_vector)
			steering_force = normalise(steering_force)*self.max_force
			self.direction = steering_force
			return(steering_force)
			#self.apply_force(steering_force)

		def eat(self, list_of_stuff, index):
			closest = None
			closest_distance = max(game_width, game_height)
			bot_x = self.core_position[0]
			bot_y = self.core_position[1]
			item_number = len(list_of_stuff)-1

			for i in list_of_stuff[::-1]:
				item_x = i[0]
				item_y = i[1]
				distance = math.hypot(bot_x-item_x, bot_y-item_y)

				if distance < 1:
					list_of_stuff.pop(item_number)
					self.ate_counter += 1
					if index == 1:
						print(self.name,"ate poison")
					self.health += nutrition[index]

				if distance < closest_distance:
					closest_distance = distance
					closest = i
				item_number -=1
			#if it finds closet food or poison to react to	
			if closest_distance < self.dna[2 + index]:
				seek = self.seek(closest) # index)
				seek *= self.dna[index]
				seek = normalise(seek)*self.max_force
				self.apply_force(seek)


		def boundaries(self):
			desired = None
			x_pos = self.core_position[0]
			y_pos = self.core_position[1]
			#print(x_pos,"x",y_pos,"y",boundary_size,"boundary_size",game_height,"game height",game_width,"game_width")
			if x_pos < boundary_size:
				#print("turn back from left side")
				desired = numpy.array([self.max_vel, self.direction_vector[1]])
				steer = desired-self.direction_vector
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			elif x_pos > (game_width - boundary_size):
				#print("turn back from right side")
				desired = numpy.array([-self.max_vel, self.direction_vector[1]])
				steer = desired-self.direction_vector
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			if y_pos < boundary_size:
				#print("turn back from top")
				desired = numpy.array([self.direction_vector[0], self.max_vel])
				steer = desired-self.direction_vector
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			elif y_pos > (game_height - boundary_size):
				#print("turn back from bottom")
				desired = numpy.array([self.direction_vector[0], -self.max_vel])
				steer = desired-self.direction_vector
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			'''if desired != None:
				steer = desired-self.direction_vector
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)'''


		def draw_bot(self):
			#body
			#pygame.gfxdraw.aacircle(gameDisplay, int(self.core_position[0]), int(self.core_position[1]), 10, self.colour)
			pygame.gfxdraw.filled_circle(gameDisplay, int(self.core_position[0]), int(self.core_position[1]), abs(int((self.health//10)+1)), self.colour)

			#vision
			#pygame.draw.circle(gameDisplay, green, (self.core_position), abs(int(self.dna[2])), abs(int(min(2, self.dna[2]))))
			#pygame.draw.circle(gameDisplay, blue, (self.core_position), abs(int(self.dna[3])), abs(int(min(2, self.dna[3]))))
			#screen.blit(pygame.transform.scale(sprite_surface,(len(image[0])*3,len(image)*3)),(10,10))	
			
			#direction
			#nose
			#pygame.draw.line(gameDisplay, white, self.core_position, self.core_position + (self.direction_vector * 7), 5)
			
			#legs
			# scaled_to_force_vector = self.direction_vector*magnitude_calc(self.max_force)*5
			# scaled_to_max_health = self.direction_vector*self.max_health*.05 #scaling it down
			# offset_against_direction = self.core_position - scaled_to_max_health
			# right_side_knee = offset_against_direction - [scaled_to_max_health[1],-scaled_to_max_health[0]]
			# right_side_foot = right_side_knee - (scaled_to_force_vector*2+3)
			# left_side_knee = offset_against_direction - [-scaled_to_max_health[1],scaled_to_max_health[0]]
			# left_side_foot = left_side_knee - (scaled_to_force_vector*2+3)
			#print(magnitude_calc((right_side_knee-right_side_foot)))

			#180-(180*y1/abs(y1))+(math.degrees(math.atan(y1/x1)))+90-(90*x1/abs(x1))
			# font_word = pygame.font.Font("freesansbold.ttf",24)
			degree_angle = math.degrees(math.atan(self.direction_vector[1]/self.direction_vector[0]))+90-(90*self.direction_vector[0]/abs(self.direction_vector[0]))
			print(degree_angle,"init degree")
			if degree_angle < 0: 
				degree_angle = 360+degree_angle
			
			#text_word = font_word.render(str(degree_angle),True,green,blue)
			#rect = text_word.get_rect()
			#print(degree_angle,"angle|\t",math.atan(self.direction_vector[1]/self.direction_vector[0]),"radian\t",self.direction_vector,"d_vector|\t",self.direction,"directio")
			# rect.center = (self.core_position + (self.direction_vector * 7))
			# gameDisplay.blit(text_word,rect)

			# self.sprite_surface = pygame.transform.rotate(self.sprite_surface_og,-(degree_angle)).copy()
			# temp_center = self.sprite_surface.get_rect().center
			# self.sprite_surface.set_colorkey((255, 255, 255))
			if math.isnan(degree_angle):
				iter=0
			else:
				iter = int(round(degree_angle/18,0))
			if iter == 20: iter = 0 
			temp_surface = self.sprite_surface[iter]
			temp_offset = [(pygame.Surface.get_width(temp_surface))/2,(pygame.Surface.get_height(temp_surface))/2]

			#gameDisplay.blit(temp_surface,(self.core_position[0]-self.offset_surface_for_render[0],self.core_position[1]-self.offset_surface_for_render[1]))
			gameDisplay.blit(temp_surface,(self.core_position[0]-temp_offset[0],self.core_position[1]-temp_offset[1]))

			# pygame.draw.line(gameDisplay, white, (left_side_knee), (right_side_knee), (int(self.max_vel)+2))
			# pygame.draw.line(gameDisplay, self.food_color, (left_side_knee), (left_side_foot), int(abs(magnitude_calc(scaled_to_force_vector)+2)))
			# pygame.draw.line(gameDisplay, self.poison_color, (right_side_knee), (right_side_foot), int(abs(magnitude_calc(scaled_to_force_vector)+2)))

	#attraction_rate
	
	#(max(min((1-percent_health)*255,255),0), max(min(percent_health*255,255),0), 0)



	for i in range(2):
		bots.append(create_bot(random.uniform(0,game_width),random.uniform(0,game_height)))

	running = True
	while(running):
		gameDisplay.fill(black)
		if len(bots)<2:
			bots.append(create_bot(random.uniform(0,game_width),random.uniform(0,game_height)))

		#random events
		if random.random()<0.1:
			unplaced = True
			while unplaced:
				rand_x = random.uniform(boundary_size, game_width-boundary_size)
				rand_y = random.uniform(boundary_size, game_height-boundary_size)

				good_position=True
				#check for bots
				for bot in bots:
					distance = math.hypot(rand_x-bot.core_position[0], rand_y-bot.core_position[1])
					if distance < 10:
						good_position = False
						break
				#check for other foods
				for i in food[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
						break
				#check for other foods
				for i in poison[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
						break
				# if good position then..., else start over with fresh x and y	
				if good_position == True: 
					food.append(numpy.array([rand_x, rand_y], dtype='float64'))
					unplaced=False

		if random.random()<0.01:
			unplaced = True
			while unplaced:
				rand_x = random.uniform(boundary_size, game_width-boundary_size)
				rand_y = random.uniform(boundary_size, game_height-boundary_size)

				good_position=True
				#check for bots
				for bot in bots:
					#(bot.core_position,"pos",bot.direction_vector,"d_vector")
					distance = math.hypot(rand_x-bot.core_position[0], rand_y-bot.core_position[1])
					if distance < 20:
						good_position = False
				#check for other foods
				for i in food[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
				#check for other foods
				for i in poison[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
				# if good position then..., else start over with fresh x and y	
				if good_position == True: 
					poison.append(numpy.array([rand_x, rand_y], dtype='float64'))
					unplaced=False

		if len(poison)>max_poison:
			poison.pop(0)


		pygame.event.pump()
		for event in pygame.event.get():
			if event.type == pygame.KEYDOWN or event.type == pygame.KEYUP:
				if event.key == pygame.K_SPACE:
					print("space was pressed")
					for bot in bots:
						bots.remove(bot)
			if event.type == pygame.QUIT:
				running = False
			#print(event)
		

		#print(bots[0].core_position)
		#print((bots[0].core_position),(bots[0].core_position+(-size,0)),(bots[0].core_position+(-size/2,size)))
		for bot in bots[::-1]:
			bot.eat(food, 0)
			bot.eat(poison, 1)
			bot.boundaries()
			#bot.seek(pygame.mouse.get_pos())
			bot.update()
			# if bot.age > oldest_ever:
			#  	oldest_ever = bot.age
			#  	oldest_ever_dna = bot.dna
			#  	print(oldest_ever, oldest_ever_dna)
			bot.draw_bot()
			#pygame.draw.polygon(gameDisplay, bot.colour, ((bot.core_position),tuple(map(operator.add,bot.core_position,(-size,0))),tuple(map(operator.add,bot.core_position,(-size/2,size)))))
			if bot.return_true_if_dead():
				
				if len(all_bots) % 100 == 0:
					df = pd.DataFrame([vars(f) for f in all_bots])
					df.to_csv("herbivore.csv",index=False)

				gameDisplay.blit(bot.sprite_surface[0],(10000,10000))
				bot.core_position = [10000,10000]

				if bot.reproduction_count>2:
					print("Name:",bot.name,"died","Ate:",bot.ate_counter,"Repo:",bot.reproduction_count)
				else:
					print(	"Name:",bot.name,"died")
				all_bots.append(bot)

				bots.remove(bot)
			else:
				bot.reproduce()

		#if random.random()<0.02:
			#bots.append(create_bot(random.uniform(0,game_width),random.uniform(0,game_height)))

		for i in food:
			pygame.draw.circle(gameDisplay, green, (int(i[0]), int(i[1])), 3)
		#pygame.draw.circle(gameDisplay, bot.colour, (int(self.core_position[0]), int(self.core_position[1])), 10)
		for i in poison:
			pygame.draw.circle(gameDisplay, blue, (int(i[0]), int(i[1])), 3)
		pygame.display.update()
		clock.tick(fps)


	pygame.quit()
	quit()




main()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
pd.read_csv("herbivore.csv")

,name,core_position,direction_vector,current_acceleration,colour,max_health,health,max_vel,size,accel_scaler,...,ate_counter,reproduction_count,direction,steer_food,steer_poison,percept_food,percept_poison,food_color,poison_color,dna
0,1,[243.85184071 356.87184484],[-1.88348707 -0.67266371],[-0. -0.],"(255, 0, 0)",91.338852,-0.161148,2.000000,4.566943,1.805449,...,0,0,[-0.0626884 -0.02458861],-0.170334,0.109016,39.122492,15.890645,"(217.1761560374003, 0, 0)","(0, 0, 138.9959284300559)","[-0.17033424002933356, 0.10901641445494581, 39..."
1,2,[741.21472647 49.42790102],[0.92252678 0.38593308],[0. 0.],"(255, 0, 0)",114.366199,-0.083801,1.000000,5.718310,1.545540,...,0,0,[ 0.00306699 -0.00631804],-0.025985,0.169511,85.681936,98.027895,"(33.13040889519222, 0, 0)","(0, 0, 216.12613903262022)","[-0.025984634427601738, 0.16951069728048646, 8..."
2,3,[ 85.02566538 404.34691042],[ 1.11544502 -1.66005494],[-0. 0.],"(255, 0, 0)",91.504228,-11.995772,2.000000,4.575211,1.593081,...,10,0,[-0.00777962 0.02268988],0.128212,0.068888,45.994469,98.903949,"(0, 163.46969971820718, 0)","(0, 0, 87.83157509842462)","[0.12821152919075074, 0.06888750988111736, 45...."
3,5,[517.05559003 494.72811368],[ 0.01043537 -1.99997278],[0. 0.],"(255, 0, 0)",65.626759,-0.173241,2.000000,3.281338,0.627540,...,4,0,[0.02870218 0.0019186 ],0.150415,-0.000531,15.569764,34.023144,"(0, 191.7796529400813, 0)","(0.6773152186506142, 0, 0)","[0.150415414070652, -0.0005312276224710699, 15..."
4,4,[394.38638497 50.31290774],[ 0.99571841 -0.09243839],[ 0. -0.],"(255, 0, 0)",87.695444,-0.054556,1.000000,4.384772,1.788407,...,0,0,[ 0.0112365 -0.00196351],-0.027967,0.120495,92.883133,94.944155,"(35.65769223559111, 0, 0)","(0, 0, 153.63146668967642)","[-0.027966817439679298, 0.12049526799190308, 9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,308,[526.11039595 190.70922942],[0.19798 0.3307073],[-0. -0.],"(255, 0, 0)",108.604258,-0.052437,0.385439,5.161244,0.789336,...,1,0,[-0.06667696 0.00959684],0.138826,-0.105431,74.089017,90.171916,"(0, 148.05561724058484, 0)","(175.98785488530004, 0, 0)","[0.1388264035329612, -0.10543089238979886, 74...."
296,309,[519.0135036 91.03894953],[-0.39338408 -0.29797018],[0. 0.],"(255, 0, 0)",103.143956,-0.099490,-0.493495,5.161244,1.719542,...,0,0,[ 0.01033262 -0.13453407],0.079903,-0.140033,60.611386,97.423701,"(0, 148.05561724058484, 0)","(175.98785488530004, 0, 0)","[0.07990314155561995, -0.14003273315007853, 60..."
297,310,[426.00017894 570.40776592],[-2.59242082 -1.08539821],[ 0. -0.],"(255, 0, 0)",99.886583,-0.025584,2.810469,5.161244,2.659984,...,0,0,[ 0.02072046 -0.00953593],0.098348,-0.132935,46.398589,95.070865,"(0, 148.05561724058484, 0)","(175.98785488530004, 0, 0)","[0.09834754686061065, -0.13293529474025445, 46..."
298,228,[159.57175662 542.823472 ],[-0.62862996 -0.77770455],[ 0. -0.],"(255, 0, 0)",103.224874,-0.025126,1.000000,5.161244,1.284207,...,8,8,[-0.02748157 0.02059821],0.116122,-0.138030,46.167832,85.026550,"(0, 148.05561724058484, 0)","(175.98785488530004, 0, 0)","[0.11612205273771359, -0.13802969010611768, 46..."


: 